# Signature verification with the Elliptic Curve Digital Signature Algorithm (ECDSA)

## Initialize the x and y components of the public key
The public key is a cosepublicKey with the following components:
- kty (Key Type) : Indicates the type of key, which is EC2 (Elliptic Curve key, identified by a numeric value)
- alg (Algorithm) : Specifies the hashing or encryption algorithm to be used. In this case, it’s SHA-256.
- crv (Curve) : Defines the elliptic curve being used, which is P-256 (corresponding to the curve secp256r1).
- x (X-coordinate): The x-coordinate of the public key on the elliptic curve, usually represented as a byte string.
- y (Y-coordinate): The y-coordinate of the public key on the elliptic curve, also represented as a byte string.

In [1]:
import base64
import hashlib
from sage.all import *

# x and y components of the public key (stored by the server)  
# test.designoapis iPhone pk
x_b64 = "p6x5TKlqdiR8Ua0WF82s9SO4dr-stZ2LLhxZt9v344E" 
y_b64 = "lwR_Cl8zUrAK7aJEBVlWdWv9nvcphtyVpHRlSn3Qtxg"

# burro-in-crow iPhone pk
x_b64 = "cBLsJK5C6aAOzRyccb2i-O5vs1OwnspVSr2nLYFYGe8"
y_b64 = "N0rcp3e3kDqaS_Pq0_DL7jjH2I7N1xSUZpEpHu2OIeM"

# Decode Base64-URL (remember to replace URL-safe characters)
x_bytes = base64.urlsafe_b64decode(x_b64 + '==')  # Add padding if needed
y_bytes = base64.urlsafe_b64decode(y_b64 + '==')  # Add padding if needed

# Convert bytes to integers
Qx = int.from_bytes(x_bytes, byteorder='big')
Qy = int.from_bytes(y_bytes, byteorder='big')

print("Qx:", Qx)
print("Qy:", Qy)

Qx: 50692472089539634003009937703405430140814529744798679144335567681508513159663
Qy: 25009476253988315448101442367541050843982672414675819133287889442390674383331


In [2]:
# Define curve parameters for P-256
p = 0xFFFFFFFF00000001000000000000000000000000FFFFFFFFFFFFFFFFFFFFFFFF
a = -3
b = 0x5AC635D8AA3A93E7B3EBBD55769886BC651D06B0CC53B0F63BCE3C3E27D2604B

# Verify if (Qx, Qy) satisfies the curve equation: y^2 = x^3 + ax + b mod p
lhs = (Qy * Qy) % p  # y^2 mod p
rhs = (Qx * Qx * Qx + a * Qx + b) % p  # x^3 + ax + b mod p

if lhs == rhs:
    print("The point is valid on the P-256 curve.")
else:
    print("The point is NOT valid on the P-256 curve.")


The point is valid on the P-256 curve.


## Initialize the data that was signed
The data argument is a Uint8Array that represents the concatenation of two binary buffers, authDataBuffer and clientDataHash:
- authDataBuffer : The authenticator data, which includes information such as the RPID hash, up/uv (user presence/verification) flags, and the signature counter.
- clientDataHash : The SHA-256 hash of the JSON-encoded clientData object. This object includes information like the type of request (here webauthn.get), the challenge issued by the RP, and the origin.

In [3]:
safari_clientDataJSON = 'eyJ0eXBlIjoid2ViYXV0aG4uZ2V0IiwiY2hhbGxlbmdlIjoieHF3NVpJbHdIVklZVU50OWNGUmM3TzJMZzc3Z21vczFidjFoQlVSMHFCQSIsIm9yaWdpbiI6Imh0dHBzOi8vYnVycm8taW4tY3Jvdy5uZ3Jvay1mcmVlLmFwcCJ9'
# = {"type":"webauthn.get","challenge":"xqw5ZIlwHVIYUNt9cFRc7O2Lg77gmos1bv1hBUR0qBA","origin":"https://burro-in-crow.ngrok-free.app"}
iOS_clientDataJSON = 'eyJvcmlnaW4iOiJodHRwczovL2J1cnJvLWluLWNyb3cubmdyb2stZnJlZS5hcHAiLCJ0eXBlIjoid2ViYXV0aG4uZ2V0IiwiY2hhbGxlbmdlIjoiLTRoekhGQmhGNVNXYkNBLVNvRFI3OUVtNDlMcXA4aG1UbS1zSkctd0dJayJ9'
# = {"origin":"https://burro-in-crow.ngrok-free.app","type":"webauthn.get","challenge":"-4hzHFBhF5SWbCA-SoDR79Em49Lqp8hmTm-sJG-wGIk"}

authenticatorData = 'yKef6PtY1VxKUCHfXlnimzTvvN3IE3B4Uz2G2O6WMI0dAAAAAA'

# Concated Uint8 data Array
safari_data = [
    200,167,159,232,251,88,213,92,74,80,33,223,94,89,226,155,52,239,
    188,221,200,19,112,120,83,61,134,216,238,150,48,141,29,0,0,0,0,
    234,182,144,75,170,128,174,211,176,156,97,182,186,141,176,248,
    37,228,235,85,2,3,17,3,191,74,85,152,106,152,211,243
]
ios_data = [
    200,167,159,232,251,88,213,92,74,80,33,223,94,89,226,155,52,
    239,188,221,200,19,112,120,83,61,134,216,238,150,48,141,29,
    0,0,0,0,62,54,237,73,145,168,185,196,109,95,200,35,190,171,
    144,232,64,3,96,68,98,80,162,146,18,60,232,224,163,2,48,17
]

# Convert to bytes
data_bytes_safari = bytes(safari_data)
data_bytes_ios = bytes(ios_data)

# Hash using SHA-256
z_safari = hashlib.sha256(data_bytes_safari).digest()
z_ios = hashlib.sha256(data_bytes_ios).digest()

# Convert hash to integer
z_int_safari = int.from_bytes(z_safari, byteorder='big')
z_int_ios = int.from_bytes(z_ios, byteorder='big')

print("Hashed data (z) from safari =", z_int_safari)
print("Hashed data (z) from iOS =", z_int_ios)

Hashed data (z) from safari = 111663185585327941903865263811186884946990535701971915813997644337257079683405
Hashed data (z) from iOS = 35718193778528203138044231854073846732336498710625396111975154180590011193389


## Initialize the signature

In [4]:
# Signature bytes (Uint8 array) -- (See below how the signature is unwrapped)
safari_unwrapped_signature = [
    254,61,52,151,150,221,132,175,106,61,221,75,100,213,100,178,
    96,10,178,83,234,56,64,138,120,42,147,172,208,242,161,159,187,
    145,240,100,135,192,114,115,51,221,96,62,207,210,136,215,12,
    23,192,167,220,37,100,105,183,225,172,151,157,243,40,206
]
ios_unwrapped_signature = [
    143,79,59,170,108,82,25,120,164,240,79,252,105,160,57,167,
    238,26,8,80,253,179,145,67,121,160,132,138,122,134,83,141,
    177,205,117,56,170,219,127,28,30,233,174,8,41,163,18,53,
    131,250,222,186,63,56,55,192,235,30,121,141,242,52,155,173
]

# Extract r and s
r_bytes_safari = safari_unwrapped_signature[0:32]  # First 32 bytes
s_bytes_safari = safari_unwrapped_signature[32:64]  # Last 32 bytes

r_bytes_ios = ios_unwrapped_signature[0:32]  # First 32 bytes
s_bytes_ios = ios_unwrapped_signature[32:64]  # Last 32 bytes

# Convert to integers
r_safari = int.from_bytes(r_bytes_safari, byteorder='big')
s_safari = int.from_bytes(s_bytes_safari, byteorder='big')

r_ios = int.from_bytes(r_bytes_ios, byteorder='big')
s_ios = int.from_bytes(s_bytes_ios, byteorder='big')

# Output r and s
print("Safari r =", r_safari)
print("Safari s =", s_safari)

print("iOS r =", r_ios)
print("iOS s =", s_ios)

Safari r = 114995604188751095033987731828442252603831398966687574270096017760901632532895
Safari s = 84840354638877904312056263055551817568975002085777108753838331394550582683854
iOS r = 64820730063157481232671520420797418652915726358766979194211244803405268800397
iOS s = 80422386879590692601858417900138395609755002051197423520359900143505177353133


## Define the elliptic curve
## P-256 prime field Weierstrass curve from the Standard curve database (std)

In [5]:
p = 0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff
K = GF(p)
a = K(0xffffffff00000001000000000000000000000000fffffffffffffffffffffffc)
b = K(0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b)
E = EllipticCurve(K, (a, b))
Gx = 0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296
Gy = 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5
G = E(Gx, Gy)
E.set_order(0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551 * 0x1)
# Curve order
n = 0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551

### Explenation of the coefficients:
- $p$ : Prime number that defines the finite field $F_p$ over which the elliptic curve is defined. In this case, it’s a 256-bit prime
- $K$ This represents the finite field $F_p$ The command GF(p) creates a Galois field (finite field) with p elements. This field is used for arithmetic operations on the elliptic curve.
- $b$ : This is the coefficient a of the elliptic curve equation 
$$ y^2 = x^3 + ax + b $$
The value given is the specific constant for the P-256 curve
- $b$ : This is the coefficient $b$ of the elliptic curve. Like $a$, this is also a specific constant defined for the P-256 curve.
- $E$ : This creates an instance of an elliptic curve defined by the equation $$ y^2 = x^3 + ax + b $$ over the finite field $K$. So, E represents the P-256 elliptic curve itself.
- $G_x$ : This is the x-coordinate of the generator point $G$ on the elliptic curve. The generator point is a special point used to generate other points on the curve, important for cryptographic operations.
- $G_y$ : This is the y-coordinate of the generator point $G$.
- $G$ : This defines the generator point $G$ on the elliptic curve $E$ using the coordinates $(G_x, G_y)$
- E.set_order(...) : This sets the order of the elliptic curve $E$ to the specified value. The order of the curve is the number of points on $0x1$ does not change the value; it’s often included for clarity.
- $n$ : This is the order $n$ of the generator point $G$ on the elliptic curve. It indicates how many times you can add the point $G$ to itself before returning to the identity element (the point at infinity).

# Signature verification

In [6]:
# Public key as a point on the elliptic curve
Q = E(Qx, Qy) # The same pk is used both on safari and on the iOS app (same passkey)

# Ensure signature r and s components are Integers
r_safari = Integer(r_safari)
s_safari = Integer(s_safari)
r_ios = Integer(r_ios)
s_ios = Integer(s_ios)

#https://github.com/bitcoin/bips/blob/master/bip-0062.mediawiki#low-s-values-in-signatures
if not (0x1 <= s_safari <= 0x7FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF5D576E7357A4501DDFE92F46681B20A0):
    print("S (safari) is not in between 0x1 and 0x7FFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF 5D576E73 57A4501D DFE92F46 681B20A0")
    #s_ios = Integer(0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141 - s_ios)
else:
    print("S (safari) is in between 0x1 and 0x7FFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF 5D576E73 57A4501D DFE92F46 681B20A0")
    
if not (0x1 <= s_ios <= 0x7FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF5D576E7357A4501DDFE92F46681B20A0):
    print("S (iOS) is not in between 0x1 and 0x7FFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF 5D576E73 57A4501D DFE92F46 681B20A0")
    #s_ios = Integer(0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141 - s_ios)
else:
    print("S (iOS) is in between 0x1 and 0x7FFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF 5D576E73 57A4501D DFE92F46 681B20A0")


print()

# Compute w, u1, and u2
w_safari = inverse_mod(s_safari, n)
w_ios = inverse_mod(s_ios, n)

u1_safari = (z_int_safari * w_safari) % n
u2_safari = (Integer(r_safari) * w_safari) % n
u1_ios = (z_int_ios * w_ios) % n
u2_ios = (Integer(r_ios) * w_ios) % n

# Compute point R
R_safari = u1_safari * G + u2_safari * Q
R_ios = u1_ios * G + u2_ios * Q

# Verify the signature
if R_safari == E(0):
    print("Invalid signature (R is point at infinity)")
else:
    # Check if R is a valid point on the curve
    if R_safari not in E:
        print("R is not a valid point on the curve.")
    else:
        print("R[0]", R_safari[0])
        print("n", n)
        r_prime = Integer(R_safari[0]) % n
        if r_prime == r_safari:
            print("r_prime", r_prime)
            print("r", r_safari)
            print("Safari Signature is valid")
        else:
            print("r_prime", r_prime)
            print("r", r_safari)
            print("Safari Signature is invalid")

print()

if R_ios == E(0):
    print("Invalid signature (R is point at infinity)")
else:
    # Check if R is a valid point on the curve
    if R_ios not in E:
        print("R is not a valid point on the curve.")
    else:
        print("R[0]", R_ios[0])
        print("n", n)
        r_prime = Integer(R_ios[0]) % n
        if r_prime == r_ios:
            print("r_prime", r_prime)
            print("r", r_ios)
            print("iOS Signature is valid")
        else:
            print("r_prime", r_prime)
            print("r", r_ios)
            print("iOS Signature is invalid")

S (safari) is not in between 0x1 and 0x7FFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF 5D576E73 57A4501D DFE92F46 681B20A0
S (iOS) is not in between 0x1 and 0x7FFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF 5D576E73 57A4501D DFE92F46 681B20A0

R[0] 114995604188751095033987731828442252603831398966687574270096017760901632532895
n 115792089210356248762697446949407573529996955224135760342422259061068512044369
r_prime 114995604188751095033987731828442252603831398966687574270096017760901632532895
r 114995604188751095033987731828442252603831398966687574270096017760901632532895
Safari Signature is valid

R[0] 9128802220445562263621278740890376524270608804478266199226378645549974763625
n 115792089210356248762697446949407573529996955224135760342422259061068512044369
r_prime 9128802220445562263621278740890376524270608804478266199226378645549974763625
r 64820730063157481232671520420797418652915726358766979194211244803405268800397
iOS Signature is invalid


## Let's try to find the issue in the signature
- $w$ (Inverse of the signature's 's' value) : This is the modular inverse of the s value of the signature, computed as $w=s^{-1}\mod n$, where s is one part of the signature and n is the order of the elliptic curve.
It is used to compute the weights ($u_1$ and $u_2$) for combining points on the elliptic curve during the verification process.
- $u_1$ (First combination factor) : $u_1=z\cdot w\mod n$, where $z$ is the hash of the message, $w$ is the inverse of $s$, and $n$ is the order of the elliptic curve.
This value represents a weight applied to the generator point $G$ on the elliptic curve. It is one of the factors used to compute a linear combination of points.
- $u_2$ (Second combination factor) : $u_2=r\cdot w\mod n$, where $r$ is the other part of the signature, and $w$ is the inverse of $s$.
This value represents a weight applied to the public key point $Q$ on the elliptic curve.
- $R$ (Computed point on the elliptic curve) : $R=u_1\cdot G + u_2\cdot Q$ where G is the generator point, and Q is the public key.
This is the computed point that is used to derive the value r', which is compared to the original r value in the signature. If $r' \equiv r\mod n$, the signature is valid. Otherwise, it's invalid.

In [7]:
from hashlib import sha256

print("Safari values:")
print("w:", w_safari)
print("u1:", u1_safari)
print("u2:", u2_safari)
print("R:", R_safari)
print("R[0] mod n:", Integer(R_safari[0]) % n)
print("r:", r_safari)
if not (1 <= r_safari < n) or not (1 <= r_safari < n):
    print("Invalid r or s values")
else: 
    print("Valid r & s values")

print()

print("iOS values:")
print("w:", w_ios)
print("u1:", u1_ios)
print("u2:", u2_ios)
print("R:", R_ios)
print("R[0] mod n:", Integer(R_ios[0]) % n)
print("r:", r_ios)
if not (1 <= r_ios < n) or not (1 <= r_ios < n):
    print("Invalid r or s values")
else: 
    print("Valid r & s values")

print("\nHashed messages:")

hashed_message_safari = sha256(data_bytes_safari).hexdigest()
z_safari = int(hashed_message_safari, 16)
print(z_safari)

hashed_message_ios = sha256(data_bytes_ios).hexdigest()
z_ios = int(hashed_message_ios, 16)
print(z_ios)

Safari values:
w: 109346522389849262490409418703911117510622828348467504612840171930629426944308
u1: 81277821092030767349912286321739541153247543325524113089465806133250608666176
u2: 58918156613513959957695615028754320012069087903006837354768258255713551470197
R: (114995604188751095033987731828442252603831398966687574270096017760901632532895 : 65859002414739337964010061183138416506525448958241162965337493179724067595815 : 1)
R[0] mod n: 114995604188751095033987731828442252603831398966687574270096017760901632532895
r: 114995604188751095033987731828442252603831398966687574270096017760901632532895
Valid r & s values

iOS values:
w: 22893096412854623541226216327211861737887465008243965617868280927835207797391
u1: 27217719173103029063865869460655755134225378281481437768184971403567659496383
u2: 86104692930799735326670333870685127441527805395212411033366986324934462867611
R: (9128802220445562263621278740890376524270608804478266199226378645549974763625 : 13962472821358556848286406995384727063

# Signature Formats for Packed Attestation, FIDO U2F Attestation, and Assertion Signatures

For COSEAlgorithmIdentifier -7 (ES256), and other ECDSA-based algorithms, the sig value MUST be encoded as an ASN.1 DER Ecdsa-Sig-Value, as defined in [RFC3279] [section 2.2.3.](https://www.w3.org/TR/webauthn-2/#sctn-signature-attestation-types)

## ASN.1 signature format
     Example:
        30 44                                ; SEQUENCE (68 Bytes)
            02 20                            ; INTEGER (32 Bytes)
            |  3d 46 28 7b 8c 6e 8c 8c  26 1c 1b 88 f2 73 b0 9a
            |  32 a6 cf 28 09 fd 6e 30  d5 a7 9f 26 37 00 8f 54
            02 20                            ; INTEGER (32 Bytes)
            |  4e 72 23 6e a3 90 a9 a1  7b cf 5f 7a 09 d6 3a b2
            |  17 6c 92 bb 8e 36 c0 41  98 a2 7b 90 9b 6e 8f 13

## DER encoding
For reference, here is how to encode signatures correctly in [DER format](https://github.com/bitcoin/bips/blob/master/bip-0062.mediawiki#low-s-values-in-signatures). (48 = 0x30)

**0x30 [total-length] 0x02 [R-length] [R] 0x02 [S-length] [S] [sighash-type]**
* total-length: 1-byte length descriptor of everything that follows, excluding the sighash byte.
* R-length: 1-byte length descriptor of the R value that follows.
* R: arbitrary-length big-endian encoded R value. It cannot start with any 0x00 bytes, unless the first byte that follows is 0x80 or higher, in which case a single 0x00 is required.
* S-length: 1-byte length descriptor of the S value that follows.
* S: arbitrary-length big-endian encoded S value. The same rules apply as for R.
* sighash-type: 1-byte hashtype flag (only 0x01, 0x02, 0x03, 0x81, 0x82 and 0x83 are allowed).
This is already enforced by the reference client as of version 0.8.0 (only as relay policy, not as a consensus rule).

This rule, combined with the low S requirement above, results in S-length being at most 32 (and R-length at most 33), and the total signature size being at most 72 bytes (and on average 71.494 bytes).


#### Safari (success):
**signature :**\
48,70,\
2,33,\
0,254,61,52,151,150,221,132,175,106,61,221,75,100,213,100,178,96,10,178,83,234,56,64,138,120,42,147,172,208,242,161,159,\
2,33,\
0,187,145,240,100,135,192,114,115,51,221,96,62,207,210,136,215,12,23,192,167,220,37,100,105,183,225,172,151,157,243,40,206

0x30, 0x46,\
0x02, 0x21,\
0x00, 0xFE, 3D, 34, 97, 96, DD, 84, AF, 6A, 3D, DD, 4B, 64, D5, 64, B2, 60, 0A, B2, 53, EA, 38, 40, 8A, 78, 2A, 93, AC, D0, F2, A1, 9F,\
0x02, 0x21,\
0x00, 0xBB, 91, F0, 64, 87, C0, 72, 73, 33, DD, 60, 3E, CF, D2, 88, D7, 0C, 17, C0, A7, DC, 25, 64, 69, B7, E1, AC, 97, 9D, F3, 28, CE

**unwrappedSignature :**\
254,61,52,151,150,221,132,175,106,61,221,75,100,213,100,178,96,10,178,83,234,56,64,138,120,42,147,172,208,242,161,159,\
187,145,240,100,135,192,114,115,51,221,96,62,207,210,136,215,12,23,192,167,220,37,100,105,183,225,172,151,157,243,40,206

0xFE, 3D, 34, 97, 96, DD, 84, AF, 6A, 3D, DD, 4B, 64, D5, 64, B2, 60, 0A, B2, 53, EA, 38, 40, 8A, 78, 2A, 93, AC, D0, F2, A1, 9F, 
0xBB, 91, F0, 64, 87, C0, 72, 73, 33, DD, 60, 3E, CF, D2, 88, D7, 0C, 17, C0, A7, DC, 25, 64, 69, B7, E1, AC, 97, 9D, F3, 28, CE

#### iOS (failure):
**signature:**\
48,70,\
2,33,\
0,143,79,59,170,108,82,25,120,164,240,79,252,105,160,57,167,238,26,8,80,253,179,145,67,121,160,132,138,122,134,83,141,\
2,33,\
0,177,205,117,56,170,219,127,28,30,233,174,8,41,163,18,53,131,250,222,186,63,56,55,192,235,30,121,141,242,52,155,173

0x30, 0x46,\
0x02, 0x21,\
0x00, 0x8F, 4F, 3B, AA, 6C, 52, 19, 78, A4, F0, 4F, FC, 69, A0, 39, A7, EE, 1A, 08, 50, FD, B3, 91, 43, 79, A0, 84, 8A, 7A, 86, 53, 8D,\
0x02, 0x21,\
0x00, 0xB1, CD, 75, 38, AA, DB, 7F, 1C, 1E, E9, AE, 08, 29, A3, 12, 35, 83, FA, DE, BA, 3F, 38, 37, C0, EB, 1E, 79, 8D, F2, 34, 9B, AD

**unwrappedSignature:**\
143,79,59,170,108,82,25,120,164,240,79,252,105,160,57,167,238,26,8,80,253,179,145,67,121,160,132,138,122,134,83,141,\
177,205,117,56,170,219,127,28,30,233,174,8,41,163,18,53,131,250,222,186,63,56,55,192,235,30,121,141,242,52,155,173

0x8F, 4F, 3B, AA, 6C, 52, 19, 78, A4, F0, 4F, FC, 69, A0, 39, A7, EE, 1A, 08, 50, FD, B3, 91, 43, 79, A0, 84, 8A, 7A, 86, 53, 8D,\
0xB1, CD, 75, 38, AA, DB, 7F, 1C, 1E, E9, AE, 08, 29, A3, 12, 35, 83, FA, DE, BA, 3F, 38, 37, C0, EB, 1E, 79, 8D, F2, 34, 9B, AD

##### We can notice that r or s parts with 33 bytes have a padding byte (0x00) at the begining that is removed in the unwrappedSignature, as r and s should have a size of 32 bytes.

In [8]:
import base64
# The signature sent from the iOS app
signature =  "MEYCIQCPTzuqbFIZeKTwT_xpoDmn7hoIUP2zkUN5oISKeoZTjQIhALHNdTiq238cHumuCCmjEjWD-t66Pzg3wOseeY3yNJut"
# The raw signatured converted to a Uint8 array on the server
raw_signature = [48,70,2,33,0,143,79,59,170,108,82,25,120,164,240,79,252,105,160,57,167,238,26,8,80,253,179,145,67,121,160,132,138,122,134,83,141,2,33,0,177,205,117,56,170,219,127,28,30,233,174,8,41,163,18,53,131,250,222,186,63,56,55,192,235,30,121,141,242,52,155,173]
raw_signature_bytes = bytes(raw_signature)
b64encoded_signature = base64.urlsafe_b64encode(raw_signature_bytes)

print(b64encoded_signature.decode())
print("The signature wasn't tempered in transit:",b64encoded_signature.decode() == signature)

MEYCIQCPTzuqbFIZeKTwT_xpoDmn7hoIUP2zkUN5oISKeoZTjQIhALHNdTiq238cHumuCCmjEjWD-t66Pzg3wOseeY3yNJut
The signature wasn't tempered in transit: True


## Iterations on mulitple signatures

#### Safari (success):
1) **signature:**\
48,69,\
2,32,\
99,138,158,52,152,36,26,77,56,14,252,142,192,223,47,197,148,129,74,158,72,157,24,53,132,71,10,199,199,18,187,218,\
2,33,\
0,207,225,120,159,32,111,253,172,227,125,49,223,184,169,235,73,139,164,218,25,120,8,202,190,219,83,91,190,237,250,191,33\
**unwrappedSignature:**\
99,138,158,52,152,36,26,77,56,14,252,142,192,223,47,197,148,129,74,158,72,157,24,53,132,71,10,199,199,18,187,218,\
207,225,120,159,32,111,253,172,227,125,49,223,184,169,235,73,139,164,218,25,120,8,202,190,219,83,91,190,237,250,191,33

2) **signature:**\
48,69,\
2,32,\
75,9,212,218,113,244,110,124,82,157,111,53,228,225,102,211,27,33,237,76,201,221,97,193,187,254,125,219,9,122,250,43,\
2,33,\
0,243,177,25,66,0,227,136,157,17,139,199,185,245,158,218,64,226,158,148,108,88,24,8,24,52,155,213,146,54,143,106,27\
**unwrappedSignature:**\
75,9,212,218,113,244,110,124,82,157,111,53,228,225,102,211,27,33,237,76,201,221,97,193,187,254,125,219,9,122,250,43,\
243,177,25,66,0,227,136,157,17,139,199,185,245,158,218,64,226,158,148,108,88,24,8,24,52,155,213,146,54,143,106,27

3) **signature:**\
48,70,\
2,33,\
0,254,243,44,70,78,145,154,188,110,244,108,222,77,78,65,15,164,84,227,49,51,148,208,161,57,85,213,194,95,239,169,85,\
2,33,\
0,193,155,245,110,17,12,22,125,39,161,76,246,145,101,129,221,252,70,134,110,221,6,230,188,214,136,15,63,60,124,95,98\
**unwrappedSignature:**\
254,243,44,70,78,145,154,188,110,244,108,222,77,78,65,15,164,84,227,49,51,148,208,161,57,85,213,194,95,239,169,85,\
193,155,245,110,17,12,22,125,39,161,76,246,145,101,129,221,252,70,134,110,221,6,230,188,214,136,15,63,60,124,95,98

4) **signature:**\
48,70,\
2,33,\
0,141,14,142,89,211,56,105,85,202,169,185,93,213,131,200,205,8,111,17,188,192,123,212,123,221,19,94,5,163,43,243,64,\
2,33,\
0,146,198,11,217,80,30,152,162,49,177,106,97,250,199,151,252,47,216,70,144,113,71,181,23,239,172,38,93,11,146,183,197\
**unwrappedSignature:**\
141,14,142,89,211,56,105,85,202,169,185,93,213,131,200,205,8,111,17,188,192,123,212,123,221,19,94,5,163,43,243,64,\
146,198,11,217,80,30,152,162,49,177,106,97,250,199,151,252,47,216,70,144,113,71,181,23,239,172,38,93,11,146,183,197

5) **signature:**\
48,68,\
2,32,\
69,110,156,68,24,39,169,204,106,52,216,93,49,40,86,145,38,69,91,160,53,102,200,67,96,189,212,232,58,55,2,168,\
2,32,\
35,58,112,165,215,60,80,62,83,69,53,245,202,114,168,173,116,19,224,134,231,146,175,13,219,60,215,164,41,221,152,222\
**unwrappedSignature:**\
69,110,156,68,24,39,169,204,106,52,216,93,49,40,86,145,38,69,91,160,53,102,200,67,96,189,212,232,58,55,2,168,\
35,58,112,165,215,60,80,62,83,69,53,245,202,114,168,173,116,19,224,134,231,146,175,13,219,60,215,164,41,221,152,222


#### iOS (failure):

1) **signature:**\
48,69,\
2,33,\
0,143,201,124,214,5,63,31,19,41,74,114,187,225,161,232,178,17,173,203,243,210,8,184,138,13,150,24,49,32,52,37,228,\
2,32,\
106,57,148,47,207,122,9,100,32,116,250,134,25,149,237,239,198,156,200,132,26,236,8,71,90,95,41,70,198,48,178,83\
**unwrappedSignature:**\
143,201,124,214,5,63,31,19,41,74,114,187,225,161,232,178,17,173,203,243,210,8,184,138,13,150,24,49,32,52,37,228,\
106,57,148,47,207,122,9,100,32,116,250,134,25,149,237,239,198,156,200,132,26,236,8,71,90,95,41,70,198,48,178,83

2) **signature:**\
48,70,\
2,33,\
0,184,177,44,116,153,145,27,22,212,240,186,104,45,82,162,146,226,5,163,240,79,76,38,158,14,186,18,80,229,200,236,94,\
2,33,\
0,167,109,255,248,190,113,117,132,225,177,64,146,149,249,215,210,66,43,10,221,0,44,215,106,195,26,104,62,184,29,78,163\
**unwrappedSignature:**\
184,177,44,116,153,145,27,22,212,240,186,104,45,82,162,146,226,5,163,240,79,76,38,158,14,186,18,80,229,200,236,94,\
167,109,255,248,190,113,117,132,225,177,64,146,149,249,215,210,66,43,10,221,0,44,215,106,195,26,104,62,184,29,78,163

3) **signature:**\
48,68,\
2,32,\
28,98,32,218,244,127,249,220,106,141,241,107,78,99,173,169,71,126,199,132,104,89,7,94,157,229,203,72,28,37,36,146,\
2,32,\
83,3,222,182,121,170,76,9,1,48,247,201,178,98,189,135,182,69,230,28,154,29,68,28,17,72,50,151,166,183,88,153\
**unwrappedSignature:**\
28,98,32,218,244,127,249,220,106,141,241,107,78,99,173,169,71,126,199,132,104,89,7,94,157,229,203,72,28,37,36,146,\
83,3,222,182,121,170,76,9,1,48,247,201,178,98,189,135,182,69,230,28,154,29,68,28,17,72,50,151,166,183,88,153

4) **signature:**\
48,69,\
2,32,\
70,206,82,66,240,88,240,18,126,203,253,216,78,209,212,200,9,193,167,75,93,191,210,47,241,10,149,241,68,163,255,66,\
2,33,\
0,174,117,1,139,128,103,129,161,9,245,198,204,254,60,7,128,188,162,91,156,46,26,33,28,95,22,212,149,203,106,73,245\
**unwrappedSignature:**\
70,206,82,66,240,88,240,18,126,203,253,216,78,209,212,200,9,193,167,75,93,191,210,47,241,10,149,241,68,163,255,66,\
174,117,1,139,128,103,129,161,9,245,198,204,254,60,7,128,188,162,91,156,46,26,33,28,95,22,212,149,203,106,73,245

5) **signature:**\
48,69,\
2,33,\
0,234,45,115,65,183,96,183,62,224,178,9,28,232,237,105,1,193,28,240,171,102,31,214,240,133,143,139,199,215,19,66,131,\
2,32,\
71,181,227,250,149,52,69,224,172,158,235,33,254,231,81,179,76,120,132,164,203,80,207,145,222,196,42,192,3,246,53,34\
**unwrappedSignature:**\
234,45,115,65,183,96,183,62,224,178,9,28,232,237,105,1,193,28,240,171,102,31,214,240,133,143,139,199,215,19,66,131,\
71,181,227,250,149,52,69,224,172,158,235,33,254,231,81,179,76,120,132,164,203,80,207,145,222,196,42,192,3,246,53,34


In [9]:
s1_unwrapped_signature = [99,138,158,52,152,36,26,77,56,14,252,142,192,223,47,197,148,129,74,158,72,157,24,53,132,71,10,199,199,18,187,218,207,225,120,159,32,111,253,172,227,125,49,223,184,169,235,73,139,164,218,25,120,8,202,190,219,83,91,190,237,250,191,33]
s2_unwrapped_signature = [75,9,212,218,113,244,110,124,82,157,111,53,228,225,102,211,27,33,237,76,201,221,97,193,187,254,125,219,9,122,250,43,243,177,25,66,0,227,136,157,17,139,199,185,245,158,218,64,226,158,148,108,88,24,8,24,52,155,213,146,54,143,106,27]
s3_unwrapped_signature = [254,243,44,70,78,145,154,188,110,244,108,222,77,78,65,15,164,84,227,49,51,148,208,161,57,85,213,194,95,239,169,85,193,155,245,110,17,12,22,125,39,161,76,246,145,101,129,221,252,70,134,110,221,6,230,188,214,136,15,63,60,124,95,98]
s4_unwrapped_signature = [141,14,142,89,211,56,105,85,202,169,185,93,213,131,200,205,8,111,17,188,192,123,212,123,221,19,94,5,163,43,243,64,146,198,11,217,80,30,152,162,49,177,106,97,250,199,151,252,47,216,70,144,113,71,181,23,239,172,38,93,11,146,183,197]
s5_unwrapped_signature = [69,110,156,68,24,39,169,204,106,52,216,93,49,40,86,145,38,69,91,160,53,102,200,67,96,189,212,232,58,55,2,168,35,58,112,165,215,60,80,62,83,69,53,245,202,114,168,173,116,19,224,134,231,146,175,13,219,60,215,164,41,221,152,222]

i1_unwrapped_signature = [143,201,124,214,5,63,31,19,41,74,114,187,225,161,232,178,17,173,203,243,210,8,184,138,13,150,24,49,32,52,37,228,106,57,148,47,207,122,9,100,32,116,250,134,25,149,237,239,198,156,200,132,26,236,8,71,90,95,41,70,198,48,178,83]
i2_unwrapped_signature = [184,177,44,116,153,145,27,22,212,240,186,104,45,82,162,146,226,5,163,240,79,76,38,158,14,186,18,80,229,200,236,94,167,109,255,248,190,113,117,132,225,177,64,146,149,249,215,210,66,43,10,221,0,44,215,106,195,26,104,62,184,29,78,163]
i3_unwrapped_signature = [28,98,32,218,244,127,249,220,106,141,241,107,78,99,173,169,71,126,199,132,104,89,7,94,157,229,203,72,28,37,36,146,83,3,222,182,121,170,76,9,1,48,247,201,178,98,189,135,182,69,230,28,154,29,68,28,17,72,50,151,166,183,88,153]
i4_unwrapped_signature = [70,206,82,66,240,88,240,18,126,203,253,216,78,209,212,200,9,193,167,75,93,191,210,47,241,10,149,241,68,163,255,66,174,117,1,139,128,103,129,161,9,245,198,204,254,60,7,128,188,162,91,156,46,26,33,28,95,22,212,149,203,106,73,245]
i5_unwrapped_signature = [234,45,115,65,183,96,183,62,224,178,9,28,232,237,105,1,193,28,240,171,102,31,214,240,133,143,139,199,215,19,66,131,71,181,227,250,149,52,69,224,172,158,235,33,254,231,81,179,76,120,132,164,203,80,207,145,222,196,42,192,3,246,53,34]

# Extract s
sbs1 = s1_unwrapped_signature[32:64]  # Last 32 bytes (safari)
sbs2 = s2_unwrapped_signature[32:64]
sbs3 = s3_unwrapped_signature[32:64]
sbs4 = s4_unwrapped_signature[32:64]
sbs5 = s5_unwrapped_signature[32:64]

sbi1 = i1_unwrapped_signature[32:64]  # Last 32 bytes (iOS)
sbi2 = i2_unwrapped_signature[32:64]
sbi3 = i3_unwrapped_signature[32:64]
sbi4 = i4_unwrapped_signature[32:64]
sbi5 = i5_unwrapped_signature[32:64]

# Convert to integers
s_safari1 = int.from_bytes(sbs1, byteorder='big')
s_safari2 = int.from_bytes(sbs2, byteorder='big')
s_safari3 = int.from_bytes(sbs3, byteorder='big')
s_safari4 = int.from_bytes(sbs4, byteorder='big')
s_safari5 = int.from_bytes(sbs5, byteorder='big')

s_ios1 = int.from_bytes(sbi1, byteorder='big')
s_ios2 = int.from_bytes(sbi2, byteorder='big')
s_ios3 = int.from_bytes(sbi3, byteorder='big')
s_ios4 = int.from_bytes(sbi4, byteorder='big')
s_ios5 = int.from_bytes(sbi5, byteorder='big')

li_safari = [s_safari1, s_safari2, s_safari3, s_safari4, s_safari5]
print("Are safari signatures' s parts in between 0x1 and 0x7FFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF 5D576E73 57A4501D DFE92F46 681B20A0 ?")
for s in li_safari:
    if not (0x1 <= s <= 0x7FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF5D576E7357A4501DDFE92F46681B20A0):
        print("s is not in the right range")
    else:
        print("s is in the right range")

print()

li_iOS = [s_ios1, s_ios2, s_ios3, s_ios4, s_ios5]
print("Are iOS signatures' s parts in between 0x1 and 0x7FFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF 5D576E73 57A4501D DFE92F46 681B20A0 ?")
for s in li_iOS:
    if not (0x1 <= s <= 0x7FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF5D576E7357A4501DDFE92F46681B20A0):
        print("s is not in the right range")
    else:
        print("s is in the right range")
    

Are safari signatures' s parts in between 0x1 and 0x7FFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF 5D576E73 57A4501D DFE92F46 681B20A0 ?
s is not in the right range
s is not in the right range
s is not in the right range
s is not in the right range
s is in the right range

Are iOS signatures' s parts in between 0x1 and 0x7FFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF 5D576E73 57A4501D DFE92F46 681B20A0 ?
s is in the right range
s is not in the right range
s is in the right range
s is not in the right range
s is in the right range


# Potential issues
- $s$ part of the signature is not in between 0x1 and 0x7FFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF 5D576E73 57A4501D DFE92F46 681B20A0 (included) ?
- Issue with the padding?
- Encoding of the authenticatorData and the clientDataHash to produce the signature is wrong?
- Curve parameters to produce the signature are wrong?

## Ruled out issues
Upon a passkey assertion in a SwiftUI app, the `ASAuthorizationPlatformPublicKeyCredentialProvider().createCredentialAssertionRequest` method returns an `ASAuthorizationResult.passkeyAssertion`; We can then find that in the `passkeyAssertion.rawClientDataJSON` the challenge is encoded in Base64URL (instead of a plain string). So a potential issue I explored was that the signature was generated with this missencoded value which would translate in a wrong data argument. But after verifying the signature with the same encoding for the clientDataJSON, the signature was still incorrect. 

Steps to reproduce on the server:

`clientDataJSON`: eyJvcmlnaW4iOiJodHRwczovL2J1cnJvLWluLWNyb3cubmdyb2stZnJlZS5hcHAiLCJjaGFsbGVuZ2UiOiI1aGRMLWViU3pLekRsZFF2TVNON2RNUWNfR3lreTVHM296anJEOGEwOTlzIiwidHlwZSI6IndlYmF1dGhuLmdldCJ9

`decodedClientDataJSON`:\
{"origin":"https://burro-in-crow.ngrok-free.app", "challenge":"5hdL-ebSzKzDldQvMSN7dMQc_Gyky5G3ozjrD8a099s","type":"webauthn.get"}

`Base64URL_Challenge`:\
NWhkTC1lYlN6S3pEbGRRdk1TTjdkTVFjX0d5a3k1RzNvempyRDhhMDk5cw

`reencodedClientDataJSON`: eyJvcmlnaW4iOiJodHRwczovL2J1cnJvLWluLWNyb3cubmdyb2stZnJlZS5hcHAiLCJjaGFsbGVuZ2UiOiJOV2hrVEMxbFlsTjZTM3BFYkdSUmRrMVRUamRrVFZGalgwZDVhM2sxUnpOdmVtcHlSRGhoTURrNWN3IiwidHlwZSI6IndlYmF1dGhuLmdldCJ9

The `Base64URL_Challenge` corresponding to the `challenge` in `passkeyAssertion.rawClientDataJSON` returned in the SwiftUI app, a potential issue I thought of was that the signature was generated with the wrong data argument. But after testing different configurations, I was able to rule out the following potential issues and could assume the signature wasn't generated with:

- just the AuthenticatorData
- just the ClientDataJSON
- inverted signature base (`concat([ClientDataHash, AuthenticatorData])` ; instead of `concat([AuthenticatorData, ClientDataHash])`)
- normal signature base with the modified ClientDataJSON (as mentionned above): `concat([AuthenticatorData, modifiedClientDataHash])`
- just the modified ClientDataJSON
- inverted base with the modified ClientDataJSON `concat([modifiedClientDataHash, AuthenticatorData])`

In [10]:
# Signatures are generated from Safari on iOS and from an iOS app - each time with the same private/public key pair (same passkey).
iOS_version = "18.0.1"
macOS_vesrion = "15.0.1"
Xcode_vesrion = "16.0"
version()

'SageMath version 10.4, Release Date: 2024-07-19'